<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train3_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train3 = pd.read_csv(r'/content/train_data3.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train3['Target'] = train3['Target'].replace(['Others'],'Invalid')


In [ ]:
train3['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train3_Xtfidf = tfidf_vect.fit_transform(train3['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train3_Xtfidf,train3['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.668141592920354
                  precision    recall  f1-score   support

        Analysis       0.49      0.29      0.36       127
      Conclusion       0.73      0.48      0.58        46
           Facts       0.74      0.87      0.80       487
         Invalid       0.50      0.61      0.55       128
           Issue       0.61      0.53      0.57        47
Rule/Law/Holding       0.65      0.29      0.40        69

        accuracy                           0.67       904
       macro avg       0.62      0.51      0.54       904
    weighted avg       0.66      0.67      0.65       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 3, 2, 1, 3, 2, 2, 2, 5, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 5, 2, 0, 2, 2, 2, 3, 2, 1, 3, 3, 2, 2, 2, 2, 3, 5, 2, 2, 2, 3, 2, 2]
[0.9987382989222195, 0.9887895307744249, 0.998802962067503, 0.9988365426955222, 0.9937259620696538, 0.9977963105402875, 0.9986518783322484, 0.9899644511619455, 0.9872222038720243, 0.991608193957389, 0.994478124562862, 0.9980329429220735, 0.9908535613202533, 0.9986090788001315, 0.9979201690575894, 0.9974543940607021, 0.9983978391291153, 0.9933099199501206, 0.9983667114417447, 0.9831599354391891, 0.9948167586118685, 0.9980975230168365, 0.996470346424817, 0.9991959815459505, 0.998148686439996, 0.9820851872501458, 0.9888130300209466, 0.9946918581879272, 0.9953136859851264, 0.9960401237826509, 0.986627855566426, 0.982740962705975, 0.9965126605947587, 0.9971181901994156, 0.9951500871058383, 0.9978506912106749, 0.9946375510563237, 0.9979478617615427, 0.9853108077261207, 0.9981812877726235, 0.9890247277134446, 0.996097

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
7,1602,petition writ habeas corpus denied,Conclusion


In [ ]:
frame_1 = [train3,unlabel_1]
train3_1 = pd.concat(frame_1)
len(train3_1)

3535

In [ ]:
x_train_1 = tfidf_vect.transform(train3_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train3_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6605374823196606
                  precision    recall  f1-score   support

        Analysis       0.47      0.37      0.42        91
      Conclusion       0.71      0.49      0.58        35
           Facts       0.73      0.86      0.79       380
         Invalid       0.53      0.54      0.53       109
           Issue       0.67      0.39      0.49        46
Rule/Law/Holding       0.52      0.30      0.38        46

        accuracy                           0.66       707
       macro avg       0.60      0.49      0.53       707
    weighted avg       0.65      0.66      0.65       707



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 3, 1, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 5, 3, 3, 2]
[0.9983428350265039, 0.998855909339077, 0.9959882254112259, 0.9804179790552908, 0.997861173176517, 0.9981341902611077, 0.9978602595889592, 0.9987906592381196, 0.9969116184019756, 0.993147889165289, 0.9938637710369409, 0.9944450488276393, 0.9859892008388991, 0.9923467102620983, 0.9853209766136218, 0.9971096406223229, 0.9964468901044946, 0.9952585017558127, 0.9989159560130529, 0.9934839712294135, 0.9984768322997986, 0.9956472236334325, 0.9988872253078044, 0.9984290611178964, 0.9951846798058339, 0.9959323436953972, 0.9947000018234298, 0.9984660122703101, 0.9948177071685811, 0.9953667232345748, 0.9985017658456622, 0.9957847017115223, 0.9947763800130264, 0.9988717140318569, 0.9988305095619081, 0.9978460224563349, 0.9985434469695774, 0.9952582931527184, 0.9981094012235007, 0.9925344867240844, 0.9982644382546735, 0.9986022715155173,

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
5,2278,evidence state appellant shows appellant alleg...,Analysis
6,570,appellant performed work asked 7500 money ther...,Facts


In [ ]:
frame_2 = [train3_1,unlabel_2]
train3_2 = pd.concat(frame_2)
len(train3_2)

3589

In [ ]:
x_train_2 = tfidf_vect.transform(train3_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train3_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6505154639175258
                  precision    recall  f1-score   support

        Analysis       0.61      0.34      0.44       151
      Conclusion       0.75      0.46      0.57        52
           Facts       0.67      0.88      0.76       481
         Invalid       0.56      0.54      0.55       165
           Issue       0.74      0.38      0.51        60
Rule/Law/Holding       0.59      0.31      0.41        61

        accuracy                           0.65       970
       macro avg       0.65      0.49      0.54       970
    weighted avg       0.65      0.65      0.63       970



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 3, 2, 2, 2, 2, 4, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 3, 1, 5, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 0, 2, 2, 4, 2, 2, 2, 2, 2]
[0.9960014605497644, 0.9938758990259211, 0.997429112096425, 0.997264391256482, 0.9973701164481326, 0.9867204381110942, 0.9949377899842434, 0.9986898782180363, 0.998371736765329, 0.9941932071214663, 0.9904957820289495, 0.9983806451269615, 0.9985820890932438, 0.9988699815626035, 0.9987412799963519, 0.9961524582992183, 0.9840905082683646, 0.998618584982851, 0.9970151814612825, 0.9914604011405879, 0.9971466557997215, 0.9918347474721237, 0.9985769267481887, 0.988205006078848, 0.9929608713059103, 0.9986775599997594, 0.9975571230342859, 0.9979278460323134, 0.9983934194185528, 0.9891493927396842, 0.994809473031335, 0.9979550745279404, 0.9982342526801926, 0.9987247797861221, 0.997036503482332, 0.9804696622661087, 0.9984377753125658, 0.9988948527169276, 0.9983490480615712, 0.9884127973011166, 0.99

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train3_2,unlabel_3]
train3_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train3_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train3_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.644808743169399
                  precision    recall  f1-score   support

        Analysis       0.47      0.28      0.35        99
      Conclusion       0.63      0.47      0.54        40
           Facts       0.70      0.85      0.77       378
         Invalid       0.56      0.56      0.56       123
           Issue       0.61      0.44      0.51        39
Rule/Law/Holding       0.55      0.34      0.42        53

        accuracy                           0.64       732
       macro avg       0.59      0.49      0.53       732
    weighted avg       0.63      0.64      0.63       732



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 2, 5, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 3, 2, 2, 5, 3, 0, 5, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 5, 2, 2]
[0.9990676751560503, 0.99493139844251, 0.9984666543216131, 0.9955880498081698, 0.9989697742455694, 0.9952643891712495, 0.9989957075481769, 0.998608707917506, 0.9969251731645153, 0.9984509161780138, 0.997956859444022, 0.9957903694619867, 0.9989133976931732, 0.9989808259760723, 0.9986172713836086, 0.998780545891915, 0.9913559917852447, 0.9824697280860986, 0.9887261370252318, 0.9887359384816127, 0.9988538821034275, 0.9989870988277969, 0.9988791653486488, 0.9977336521269845, 0.9982643396885443, 0.9988982856232401, 0.9987757586514932, 0.9981158258470714, 0.998765970520941, 0.9929968379538598, 0.9980823184828205, 0.9833107457145648, 0.9904343964412586, 0.9983756866685536, 0.9975943656489868, 0.9938697172774252, 0.9975695428775493, 0.9961762076598318, 0.9957585719360275, 0.9984125028658877, 0.9891335939

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train3_3,unlabel_4]
train3_4 = pd.concat(frame_4)
len(train3_4)
x_train_4 = tfidf_vect.transform(train3_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train3_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6676616915422886
                  precision    recall  f1-score   support

        Analysis       0.56      0.34      0.42       146
      Conclusion       0.83      0.48      0.61        60
           Facts       0.72      0.87      0.79       532
         Invalid       0.49      0.57      0.53       137
           Issue       0.64      0.38      0.48        55
Rule/Law/Holding       0.72      0.39      0.50        75

        accuracy                           0.67      1005
       macro avg       0.66      0.51      0.55      1005
    weighted avg       0.67      0.67      0.65      1005



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 2, 3, 3, 0, 2, 3, 2, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 0, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 5, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 3, 2, 2]
[0.9908604733161255, 0.9897046853859524, 0.998951884355046, 0.9924456125053596, 0.9989788722151526, 0.9919100307912575, 0.9829584680687347, 0.9904464163234408, 0.9986928202066948, 0.9986133540825096, 0.9989788722151526, 0.996690000133391, 0.9986137999680165, 0.98945076059411, 0.9965757379985479, 0.9942954909987294, 0.9972638251296929, 0.994536873699417, 0.9962001288338054, 0.9980186162293385, 0.9988178396284575, 0.9958583584159169, 0.9983656361631501, 0.9942773351630019, 0.9918469572905747, 0.9902409830950448, 0.9975846014425921, 0.9989857459715769, 0.9987100767731298, 0.9989818250289045, 0.9989788722151526, 0.9976578458277596, 0.9961380068818608, 0.9832108053055557, 0.9957825235695202, 0.9984291518769011, 0.980723205701852, 0.9974345378251157, 0.9980186162293385, 0.982681589091983, 0.99842

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train3_4,unlabel_5]
train3_5 = pd.concat(frame_5)
len(train3_5)
x_train_5 = tfidf_vect.transform(train3_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train3_5['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6569920844327177
                  precision    recall  f1-score   support

        Analysis       0.44      0.33      0.37        95
      Conclusion       0.54      0.41      0.46        37
           Facts       0.71      0.89      0.79       387
         Invalid       0.60      0.57      0.58       127
           Issue       0.72      0.36      0.48        50
Rule/Law/Holding       0.58      0.29      0.39        62

        accuracy                           0.66       758
       macro avg       0.60      0.47      0.51       758
    weighted avg       0.64      0.66      0.63       758



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7332053742802304


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7122123340040153, 0.6107973722085158, 0.6494339030582724, None)
